In [51]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [64]:
train12_lgb = pd.read_csv('20180803\\data\\fenlei\\test_lable12_lgb.csv')
train34_lgb = pd.read_csv('20180803\\data\\fenlei\\test_lable34_lgb.csv')
train12_xgb = pd.read_csv('20180803\\data\\fenlei\\test_lable12_xgb.csv')
train34_xgb = pd.read_csv('20180803\\data\\fenlei\\test_lable34_xgb.csv')
train12_gb = pd.read_csv('20180803\\data\\fenlei\\test_lable12_gb.csv')
train34_gb = pd.read_csv('20180803\\data\\fenlei\\test_lable34_gb.csv')
train12_rf = pd.read_csv('20180803\\data\\fenlei\\test_lable12_rf.csv')
train34_rf = pd.read_csv('20180803\\data\\fenlei\\test_lable34_rf.csv')
train12_bagging = pd.read_csv('20180803\\data\\fenlei\\test_lable12_bagging.csv')
train34_bagging = pd.read_csv('20180803\\data\\fenlei\\test_lable34_bagging.csv')
train12_etc = pd.read_csv('20180803\\data\\fenlei\\test_lable12_etc.csv')
train34_etc = pd.read_csv('20180803\\data\\fenlei\\test_lable34_etc.csv')

In [69]:
demo12 = train12_lgb.merge(train12_xgb, on=['user_id'], how="left")
demo12 = demo12.merge(train12_gb, on=['user_id'], how="left")
demo12 = demo12.merge(train12_rf, on=['user_id'], how="left")
demo12 = demo12.merge(train12_bagging, on=['user_id'], how="left")
demo12 = demo12.merge(train12_etc, on=['user_id'], how="left")
demo12.columns = [
                    'user_id','lgb','xgb','gb','rf','bagging','etc'
                    ]
demo12_ = demo12.iloc[:,1:].T
toupiao = pd.DataFrame(demo12_.mean())
toupiao.columns = ['jieguo']
toupiao['user_id'] = demo12['user_id']
toupiao['jieguo_2'] = toupiao['jieguo']
del toupiao['jieguo']
toupiao.rename(columns={'jieguo_2':'zhongshu'}, inplace = True)
toupiao['zhongshu'] = toupiao.apply(lambda x: 2 if(x['zhongshu']>0.5) else 1,axis=1)
demo12 = demo12.merge(toupiao, on=['user_id'], how="left")
demo12.to_csv("toupiaojieguo12.csv",index=None)

In [67]:
demo34 = train34_lgb.merge(train34_xgb, on=['user_id'], how="left")
demo34 = demo34.merge(train34_gb, on=['user_id'], how="left")
demo34 = demo34.merge(train34_rf, on=['user_id'], how="left")
demo34 = demo34.merge(train34_bagging, on=['user_id'], how="left")
#demo34 = demo34.merge(train34_etc, on=['user_id'], how="left")
demo34.columns = [
                    'user_id','lgb','xgb','gb','rf','bagging'#,'etc'
                    ]
demo34_ = demo34.iloc[:,1:].T
toupiao = pd.DataFrame(demo34_.mean())
toupiao.columns = ['jieguo']
toupiao['user_id'] = demo34['user_id']
toupiao['jieguo_2'] = toupiao['jieguo']
del toupiao['jieguo']
toupiao.rename(columns={'jieguo_2':'zhongshu'}, inplace = True)
toupiao['zhongshu'] = toupiao.apply(lambda x: 4 if(x['zhongshu']>0.5) else 3,axis=1)
demo34 = demo34.merge(toupiao, on=['user_id'], how="left")
demo34.to_csv("toupiaojieguo34.csv",index=None)

In [72]:
lable1 = demo12[demo12['zhongshu']==1]
lable1.to_csv("toupiaojieguo1_5.csv",index=None)
print lable1.shape
lable2 = demo12[demo12['zhongshu']==2]
lable2.to_csv("toupiaojieguo2_5.csv",index=None)
print lable2.shape
lable3 = demo34[demo34['zhongshu']==3]
lable3.to_csv("toupiaojieguo3.csv",index=None)
print lable3.shape
lable4 = demo34[demo34['zhongshu']==4]
lable4.to_csv("toupiaojieguo4.csv",index=None)
print lable4.shape
demo12.shape,demo34.shape

(4146, 8)
(15403, 8)
(41, 7)
(809344, 7)


((19549, 8), (809385, 7))

In [5]:
(3760, 7)
(15789, 7)
(41, 7)
(809344, 7)

In [50]:
y['lable'][[0,1,2,5,9,10,11]]

0     1
1     0
2     1
5     0
9     1
10    1
11    0
Name: lable, dtype: int64

In [113]:
stacking_train.head(20)

,0,1
0,0.042403,0.957597
1,0.279144,0.720856
2,0.254892,0.745108
3,0.019043,0.980957
4,0.019043,0.980957
5,0.786046,0.213954
6,0.021587,0.978413
7,0.026965,0.973035
8,0.453698,0.546302
9,0.130113,0.869887


# 从这里开始是每个分类器分类

In [16]:
import pandas as pd

data_train_ = pd.read_csv("over_sampling_1.csv")
# data_train = pd.read_csv("pay_pricedengyu0.csv")
dt1=pd.to_datetime(data_train_["register_time"])
data_train_["register_time"] = dt1.dt.dayofyear
if "prediction_pay_price" in data_train_.columns:
    data_train_ = data_train_.drop("prediction_pay_price",axis=1) 
data_train_1 = data_train_.copy()
data_train_ = data_train_.iloc[:,1:]
data_train_.shape

(58233, 108)

In [17]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split   # cross_validation
from sklearn.linear_model import RidgeClassifierCV,RidgeClassifier,LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB,GaussianNB
from sklearn.neighbors import RadiusNeighborsClassifier,NearestCentroid,KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation,LabelSpreading
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis,LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
import gc
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb

In [18]:
class add_feature(BaseEstimator, TransformerMixin):
    def __init__(self, additional=1):
        self.additional = additional

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X["shaman_rate"] = X["shaman_reduce_value"]/(X["shaman_add_value"]+0.1)
        
        return X
    
    
pipe = Pipeline([
        ('add_feature', add_feature(additional=2))
    ])

data_train_ = pipe.fit_transform(data_train_)
data_train_['lable_2'] = data_train_['lable']
del data_train_['lable']
data_train_.rename(columns={'lable_2':'lable'}, inplace = True)

# data_train_.to_csv('tezhengceshi.csv',index=None)
# data_train_['pvp_lanch*pay_count>mean'].sum()

In [19]:
#data_train.drop(['user_id'],axis=1, inplace=True)
# data_train = data_train_.iloc[:,1:]
data_train = data_train_

In [20]:
x, y = data_train.iloc[:, 0:len(data_train.columns)-1], data_train.iloc[:,len(data_train.columns)-1:]
x.shape,y.columns

((58233, 108), Index([u'lable'], dtype='object'))

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.5)

In [22]:
from sklearn.model_selection import StratifiedKFold

def stacking(x_train, y_train, x_test, cv):

#     le = LabelEncoder()
#     y_train = le.fit_transform(y_train)

    clfs = [
        xgb.XGBClassifier(max_depth = 5),
        lgb.LGBMClassifier(max_depth = 5),
        RandomForestClassifier(max_depth = 6),
        GradientBoostingClassifier(max_depth = 5),
    ]

    skf = StratifiedKFold(n_splits=cv)
    skf_dataset = list(skf.split(x_train, y_train))

    # y_count: the kinds of labels
    y_count = len(set(y_train.lable))
#     print y_count
    
    # blend_train is the probabilities that every clf predicts every label (i.e. y) for every sample
    # it is used to train the clfs in the second layer
    blend_train = np.zeros((x_train.shape[0], len(clfs) * y_count))
    blend_train = pd.DataFrame(blend_train)

    # blend_test is used as the input of the clfs in the second layer to predict the labels of x_test
    blend_test = np.zeros((x_test.shape[0], len(clfs) * y_count))
    blend_test = pd.DataFrame(blend_test)

    for j, clf in enumerate(clfs):
        print j, clf

        # blend_test_j: the probabilities that j-th clf predicts every label for x_test
        blend_test_j = np.zeros((x_test.shape[0], len(skf_dataset) * y_count))
        blend_test_j = pd.DataFrame(blend_test_j)
        
        for k, (train_idx, test_idx) in enumerate(skf_dataset):
            x_train_k = x_train.iloc[train_idx,:]
            y_train_k = y_train.iloc[train_idx,:]
            x_train_holdout = x_train.iloc[test_idx,:]

            clf.fit(x_train_k, y_train_k)
            x_train_holdout_ = clf.predict_proba(x_train_holdout)
            
            #blend_train[test_idx, j*y_count: (j+1)*y_count] = clf.predict_proba(x_train_holdout)
            blend_train.iloc[test_idx,j*y_count:(j+1)*y_count] = clf.predict_proba(x_train_holdout)
            blend_test_j.iloc[:, k*y_count: (k+1)*y_count] = clf.predict_proba(x_test)

        # because there are len(skf_dataset) blend_test_j for x_test, it needs to calculated the mean value
        blend_test_j_mean = np.zeros((x_test.shape[0], y_count))
        blend_test_j_mean = pd.DataFrame(blend_test_j_mean)

        # indices: supposed y_count = 3, indices would be [0, 3, 6]
        # it is used to find the corresponding probabilities of the same label, and calculate the mean
        indices = np.arange(len(skf_dataset)) * y_count
        for c in xrange(y_count):
            blend_test_j_mean.iloc[:,c] = pd.DataFrame(blend_test_j.iloc[:, indices].mean(1))
            indices += 1
        blend_test[range(j*y_count,(j+1)*y_count)] = blend_test_j_mean
#         print blend_test_j_mean.head()
#         print blend_test.head()

#     clf = LogisticRegression()
#     clf.fit(blend_train, y_train)
#     y_pred = clf.predict(blend_test)
#     y_pred = le.inverse_transform(y_pred)

    return blend_train,blend_test

In [26]:
stacking_train,stacking_test = stacking(x,y,fit_data,4)

0 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
1 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=5, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)
2 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
 

In [27]:
stacking_train['user_id'] = data_train_1['user_id']
stacking_test['user_id'] = all_test['user_id']
stacking_train['lable'] = data_train_1['lable']
stacking_train.to_csv('stacking_train12.csv',index=None)
stacking_test.to_csv('stacking_test12.csv',index=None)

In [28]:
import pandas as pd
stacking_train12 = pd.read_csv("stacking_train12.csv")
stacking_test12 = pd.read_csv("stacking_test12.csv")
stacking_train12_ = stacking_train12.iloc[:,:8]
stacking_test12_ = stacking_test12.iloc[:,:8]
# stacking_train12_.columns,stacking_test12_.columns
lr = LogisticRegression()
lr.fit(stacking_train12_, stacking_train12.lable)
y_pred = lr.predict(stacking_test12_)
y_pred = pd.DataFrame(y_pred)
stacking_test12['lable'] = y_pred
stacking_test12__ =stacking_test12.iloc[:,[8,9]]
stacking_test12__.to_csv('stacking_test12_lr.csv',index=None)
stacking_test12__.head(),stacking_test12__.lable.sum()

(   user_id  lable
 0    14952    0.0
 1    15029    0.0
 2    15059    0.0
 3    15064    0.0
 4    15102    0.0, 5454.0)

In [29]:
stacking_test12__ = pd.read_csv('stacking_test12_lr.csv')
stacking_test12__['lable'] = stacking_test12__.apply(lambda x: 2 if(x['lable'] == 0) else 1,axis=1)   
stacking_test12__.to_csv('stacking_test12_lr.csv',index=None)

In [30]:
lable1 = stacking_test12__[stacking_test12__['lable']==2]
print lable1.shape
lable2 = stacking_test12__[stacking_test12__['lable']==1]
print lable2.shape

(14095, 2)
(5454, 2)


In [6]:
rf = RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=5, oob_score=True)
rf.fit(x_train, y_train)
print u'OOB Score=%.5f' % rf.oob_score_
y_train_pred = rf.predict(x_train)
y_test_pred = rf.predict(x_test)
print u'随机森林：', accuracy_score(y_train, y_train_pred)
print u'随机森林：', accuracy_score(y_test, y_test_pred)


OOB Score=0.99794
随机森林： 0.9979515420855278
随机森林： 0.998058371702971


In [7]:
gb = GradientBoostingClassifier(
#     n_estimators=100, learning_rate=0.1, max_depth=5
)
gb.fit(x_train, y_train)
y_train_pred = gb.predict(x_train)
y_test_pred = gb.predict(x_test)
print u'GBDT：', accuracy_score(y_train, y_train_pred)
print u'GBDT：', accuracy_score(y_test, y_test_pred)

GBDT： 0.9979951641793171
GBDT： 0.9980370057794823


In [44]:
def lightGBM(train_x,train_y,validation_x,validation_y):
#     clf = lgb.LGBMClassifier(num_leaves= 127, 
#                              max_depth=10,
#                              num_class=4,
#                              silent=True,
#                              metric='auc',
#                              n_jobs=8, 
#                              n_estimators=1000,
#                              colsample_bytree=0.9,
#                              subsample=0.9,
#                              learning_rate=0.01
#                             )
    clf = lgb.LGBMClassifier(
#                             num_leaves= 127, 
#                              max_depth=8, 
#                              silent=True,
#                              metric='auc',
#                              n_jobs=8,
#                              n_estimators=2000,
#                              colsample_bytree=0.9,
#                              subsample=0.9,
#                              learning_rate=0.02
                            )
    
    # clf.fit(train_x, train_y, **fit_params)
    clf.fit(train_x,
    train_y,
    early_stopping_rounds=100,
    eval_metric= 'auc', 
            eval_set = [(validation_x,validation_y)],
            eval_names = ['valid'],
            verbose= 100
    )
         
    return clf

clf = lightGBM(x_train,y_train, x_test, y_test)
y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)
print u'lgb：', accuracy_score(y_train, y_train_pred)
print u'lgb：', accuracy_score(y_test, y_test_pred)


Training until validation scores don't improve for 100 rounds.
[100]	valid's auc: 0.85248
Did not meet early stopping. Best iteration is:
[94]	valid's auc: 0.852651
lgb： 0.8567015782614991
lgb： 0.8089285714285714


In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)
y_train_pred = dtc.predict(x_train)
y_test_pred = dtc.predict(x_test)
print u'dtc：', accuracy_score(y_train, y_train_pred)
print u'dtc：', accuracy_score(y_test, y_test_pred)                           
                          

etc = ExtraTreesClassifier()
etc.fit(x_train, y_train)
y_train_pred = etc.predict(x_train)
y_test_pred = etc.predict(x_test)
print u'etc：', accuracy_score(y_train, y_train_pred)
print u'etc：', accuracy_score(y_test, y_test_pred)

dtc： 0.9998477677951435
dtc： 0.9957499617193871
etc： 0.9998477677951435
etc： 0.9980058471410614


In [31]:
lr = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), cv=3)
lr.fit(x_train, y_train)
print u'参数alpha=%.2f' % lr.alpha_
y_train_pred = lr.predict(x_train)
y_test_pred = lr.predict(x_test)
print u'Logistic回归：', accuracy_score(y_train, y_train_pred)
print u'Logistic回归：', accuracy_score(y_test, y_test_pred)

参数alpha=1000.00
Logistic回归： 0.9979266151747911
Logistic回归： 0.9980227618304899


In [32]:
bnb = BernoulliNB()
bnb.fit(x_train, y_train)
y_train_pred = bnb.predict(x_train)
y_test_pred = bnb.predict(x_test)
print u'bnb：', accuracy_score(y_train, y_train_pred)
print u'bnb：', accuracy_score(y_test, y_test_pred)  

bnb： 0.9089678122362643
bnb： 0.9090043123555575


In [33]:
bayes = GaussianNB() 
bayes.fit(x_train, y_train)
y_train_pred = bayes.predict(x_train)
y_test_pred = bayes.predict(x_test)
print u'bayes：', accuracy_score(y_train, y_train_pred)
print u'bayes：', accuracy_score(y_test, y_test_pred)  

bayes： 0.9763817520769458
bayes： 0.9768153022744026


In [70]:
lp = LabelPropagation()#MemoryError
lp.fit(x_train, y_train)
y_train_pred = lp.predict(x_train)
y_test_pred = lp.predict(x_test)
print u'lp：', accuracy_score(y_train, y_train_pred)
print u'lp：', accuracy_score(y_test, y_test_pred)  

lp： 0.9999034702447029
lp： 0.305984555984556


In [54]:
rc=RidgeClassifier()
rc.fit(x_train, y_train)
y_train_pred = rc.predict(x_train)
y_test_pred = rc.predict(x_test)
print u'rc：', accuracy_score(y_train, y_train_pred)
print u'rc：', accuracy_score(y_test, y_test_pred)  

rc： 0.9979266151747911
rc： 0.9980227618304899


In [86]:
rnc = RadiusNeighborsClassifier() #MemoryError
rnc.fit(x_train, y_train)
y_train_pred = rnc.predict(x_train)
y_test_pred = rnc.predict(x_test)
print u'rnc训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'rnc测试集准确率：', accuracy_score(y_test, y_test_pred) 

In [55]:
qda = QuadraticDiscriminantAnalysis() 
qda.fit(x_train, y_train)
y_train_pred = qda.predict(x_train)
y_test_pred = qda.predict(x_test)
print u'qda：', accuracy_score(y_train, y_train_pred)
print u'qda：', accuracy_score(y_test, y_test_pred)

qda： 0.958458413010423
qda： 0.9586676210112491


In [62]:
nc = NearestCentroid() 
nc.fit(x_train, y_train)
y_train_pred = nc.predict(x_train)
y_test_pred = nc.predict(x_test)
print u'nc：', accuracy_score(y_train, y_train_pred)
print u'nc：', accuracy_score(y_test, y_test_pred) 

nc： 0.9693203143639543
nc： 0.9695936201352463


In [63]:
mlp = MLPClassifier() #准确率飘忽不定
mlp.fit(x_train, y_train)
y_train_pred = mlp.predict(x_train)
y_test_pred = mlp.predict(x_test)
print u'mlp：', accuracy_score(y_train, y_train_pred)
print u'mlp：', accuracy_score(y_test, y_test_pred)  

mlp： 0.9976310532331983
mlp： 0.9977405535910776


In [70]:
lda = LinearDiscriminantAnalysis() 
lda.fit(x_train, y_train)
y_train_pred = lda.predict(x_train)
y_test_pred = lda.predict(x_test)
print u'lda：', accuracy_score(y_train, y_train_pred)
print u'lda：', accuracy_score(y_test, y_test_pred) 

lda： 0.9852619640269068
lda： 0.9854604890659887


In [110]:
ls = LabelSpreading() #MemoryError 
ls.fit(x_train, y_train)
y_train_pred = ls.predict(x_train)
y_test_pred = ls.predict(x_test)
print u'ls训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'ls测试集准确率：', accuracy_score(y_test, y_test_pred) 

ls训练集准确率： 0.9999034702447029
ls测试集准确率： 0.305984555984556


In [71]:
knn = KNeighborsClassifier() 
knn.fit(x_train, y_train)
y_train_pred = knn.predict(x_train)
y_test_pred = knn.predict(x_test)
print u'knn：', accuracy_score(y_train, y_train_pred)
print u'knn：', accuracy_score(y_test, y_test_pred) 

knn： 0.9979301761620392
knn： 0.9979818104771366


In [13]:
bagging = BaggingClassifier()
bagging.fit(x_train, y_train)
y_train_pred = bagging.predict(x_train)
y_test_pred = bagging.predict(x_test)
print u'bagging：', accuracy_score(y_train, y_train_pred)
print u'bagging：', accuracy_score(y_test, y_test_pred) 

bagging： 0.9996875233689788
bagging： 0.9979675665281442


In [7]:
Ada = AdaBoostClassifier()
Ada.fit(x_train, y_train)
y_train_pred = Ada.predict(x_train)
y_test_pred = Ada.predict(x_test)
print u'Ada：', accuracy_score(y_train, y_train_pred)
print u'Ada：', accuracy_score(y_test, y_test_pred) 

Ada： 0.9978990175236182
Ada： 0.9980094081283095


In [78]:
svc = SVC()
svc.fit(x_train, y_train)
y_train_pred = svc.predict(x_train)
y_test_pred = svc.predict(x_test)
print u'svc：', accuracy_score(y_train, y_train_pred)
print u'svc：', accuracy_score(y_test, y_test_pred) 

svc： 0.9997586756117574
svc： 0.7272200772200772


(809385, 108)
(19549, 108)


In [23]:
all_test=pd.read_csv('test_pay_pricedayu0.csv')

In [24]:
fit_data = all_test.iloc[:,1:108]
dt1=pd.to_datetime(fit_data["register_time"])
fit_data["register_time"] = dt1.dt.dayofyear
fit_data.shape

(19549, 107)

In [25]:
fit_data = pipe.fit_transform(fit_data)

In [48]:
y_pred = clf.predict(fit_data.values)
y_pred = pd.DataFrame(y_pred)
all_test['lable']=y_pred
all_test.shape

(19549, 109)

In [49]:
print all_test.columns
all_test_ =all_test.iloc[:,[0,108]]
print all_test_.columns
all_test_.to_csv('test_lable12_svc.csv',index=None)

Index([u'user_id', u'register_time', u'wood_add_value', u'wood_reduce_value',
       u'stone_add_value', u'stone_reduce_value', u'ivory_add_value',
       u'ivory_reduce_value', u'meat_add_value', u'meat_reduce_value',
       ...
       u'pvp_battle_count', u'pvp_lanch_count', u'pvp_win_count',
       u'pve_battle_count', u'pve_lanch_count', u'pve_win_count',
       u'avg_online_minutes', u'pay_price', u'pay_count', u'lable'],
      dtype='object', length=109)
Index([u'user_id', u'lable'], dtype='object')


In [83]:
lable_1 = all_test[all_test['lable'] == 1]
lable_2 = all_test[all_test['lable'] == 0]
print lable_2.shape
print lable_1.shape
all_test.shape

(0, 109)
(19549, 109)


(19549, 109)